# View Molecular Structure

In [5]:
import nglview as ng

view = ng.show_structure_file("../data/input/1aml.pdb")
view

NGLWidget()

In [6]:
import os
import subprocess
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd

class MDSimulation:
    def __init__(self, input_dir, thread_counts=[1, 2, 4, 8]):
        self.input_dir = input_dir
        self.thread_counts = thread_counts
        self.results = {}
        
    def setup_md_parameters(self):
        """Create MD parameter files"""
        # Energy minimization parameters
        em_mdp = """
; Energy minimization parameters
integrator    = steep     ; Steepest descent energy minimization
nsteps        = 50000     ; Maximum number of steps
emtol         = 1000.0    ; Stop when max force < 1000 kJ/mol/nm
emstep        = 0.01      ; Initial step-size
nstlist       = 1         ; Frequency to update neighbor list
cutoff-scheme = Verlet    ; Neighbor search method
ns_type       = grid      ; Method to determine neighbor list
coulombtype   = PME       ; Treatment of long-range electrostatic interactions
rcoulomb      = 1.0       ; Short-range electrostatic cut-off
rvdw          = 1.0       ; Short-range van der Waals cut-off
pbc           = xyz       ; Periodic boundary conditions
"""
        # NVT equilibration parameters
        nvt_mdp = """
; NVT equilibration parameters
integrator      = md        ; leap-frog integrator
nsteps          = 50000     ; 100 ps
dt              = 0.002     ; 2 fs
nstxout-compressed = 5000   ; save coordinates every 10 ps

; Temperature coupling
tcoupl          = V-rescale ; modified Berendsen thermostat
tc-grps         = Protein Non-Protein ; coupling groups
tau_t           = 0.1 0.1   ; time constant, in ps
ref_t           = 300 300   ; reference temperature, one for each group, in K

; Pressure coupling is off
pcoupl          = no        ; no pressure coupling in NVT

; Periodic boundary conditions
pbc             = xyz       ; 3-D PBC

; Constraint algorithms
constraints      = h-bonds  ; constrain H-bond lengths
constraint_algorithm = lincs ; holonomic constraints

; Neighbor searching
cutoff-scheme   = Verlet
ns_type         = grid
nstlist         = 10
rcoulomb        = 1.0
rvdw            = 1.0

; Electrostatics
coulombtype     = PME      ; Particle Mesh Ewald for long-range electrostatics
pme_order       = 4        ; cubic interpolation
fourierspacing  = 0.16     ; grid spacing for FFT
"""
        # Production MD parameters
        md_mdp = """
; Production MD parameters
integrator      = md        ; leap-frog integrator
nsteps          = 500000    ; 1000 ps (1 ns)
dt              = 0.002     ; 2 fs
nstxout-compressed = 5000   ; save coordinates every 10 ps

; Temperature coupling
tcoupl          = V-rescale ; modified Berendsen thermostat
tc-grps         = Protein Non-Protein
tau_t           = 0.1 0.1
ref_t           = 300 300

; Pressure coupling
pcoupl          = Parrinello-Rahman ; pressure coupling
pcoupltype      = isotropic         ; uniform scaling of box vectors
tau_p           = 2.0               ; time constant, in ps
ref_p           = 1.0               ; reference pressure, in bar
compressibility = 4.5e-5            ; isothermal compressibility of water

; Constraints
constraints      = h-bonds
constraint_algorithm = lincs

; Neighbor searching
cutoff-scheme   = Verlet
ns_type         = grid
nstlist         = 10
rcoulomb        = 1.0
rvdw            = 1.0

; Electrostatics
coulombtype     = PME
pme_order       = 4
fourierspacing  = 0.16
"""
        # Write parameter files
        with open(f"{self.input_dir}/em.mdp", 'w') as f:
            f.write(em_mdp)
        with open(f"{self.input_dir}/nvt.mdp", 'w') as f:
            f.write(nvt_mdp)
        with open(f"{self.input_dir}/md.mdp", 'w') as f:
            f.write(md_mdp)
            
    def run_md_simulation(self, threads):
        """Run MD simulation with specified number of threads"""
        os.environ['OMP_NUM_THREADS'] = str(threads)
        
        steps = ['em', 'nvt', 'md']
        timings = {}
        
        for step in steps:
            # Generate tpr file
            subprocess.run(['gmx_mpi', 'grompp', 
                          '-f', f'{self.input_dir}/{step}.mdp',
                          '-c', f'{self.input_dir}/ionized.gro',
                          '-p', f'{self.input_dir}/topol.top',
                          '-o', f'{step}.tpr'])
            
            # Run MD
            start_time = datetime.now()
            subprocess.run(['gmx_mpi', 'mdrun',
                          '-deffnm', step,
                          '-ntomp', str(threads),
                          '-pin', 'on'])
            end_time = datetime.now()
            
            timings[step] = (end_time - start_time).total_seconds()
            
        return timings
    
    def run_benchmarks(self):
        """Run benchmarks with different thread counts"""
        for threads in self.thread_counts:
            print(f"Running simulation with {threads} threads...")
            self.results[threads] = self.run_md_simulation(threads)
            
    def analyze_performance(self):
        """Analyze and plot performance"""
        df = pd.DataFrame(self.results).T
        
        # Plot performance for each stage
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        for idx, stage in enumerate(['em', 'nvt', 'md']):
            speedup = df[stage].iloc[0] / df[stage]
            
            axes[idx].plot(df.index, speedup, 'o-', label='Actual')
            axes[idx].plot(df.index, df.index, '--', label='Ideal')
            axes[idx].set_title(f'{stage.upper()} Speedup')
            axes[idx].set_xlabel('Number of Threads')
            axes[idx].set_ylabel('Speedup')
            axes[idx].legend()
            axes[idx].grid(True)
            
        plt.tight_layout()
        plt.show()
        
        return df

In [ ]:

# Usage example:
md_sim = MDSimulation("../data/input")
md_sim.setup_md_parameters()
md_sim.run_benchmarks()
results = md_sim.analyze_performance()

In [ ]:

# Example usage in notebook:
base_dir = "../"
md_sim = MDSimulation(base_dir)
md_sim.setup_simulation()
md_sim.run_benchmarks()
results_df = md_sim.analyze_results()
display(results_df)